# First pipeline steps
## Pipline to clean, join, filter, and check sequences before OTU picking/clustering

## Setup (run these each time you restart notebook)

# Setting up variables

## Specific variables that should not change

In [1]:
# Path to directory containing reference reads/seqs/taxonomy files
%env REF=

# Path to Trimmomatic
%env TRIMMOMATIC=

import os
ref = os.getenv("REF")

env: REF=/home/sam/data/databases/silva
env: TRIMMOMATIC=/home/sam/programs/Trimmomatic-0.39/trimmomatic-0.39.jar


## Setting up run specific variables

Replace all `<text>` (including the `<>`) below with expected paths/files/values.

In [2]:
# Set full path to output directory
base = ""

# Add path to directory containing FastQ files for analysis
%env RAW=$ref/<FastQ directory>

# Specific to clustering
%env CLUSTER_PER=<97>
%env MYREF=$ref/<Silva ref QZA>

# Reference sequences and taxa QZAs
%env REF_SEQS=$ref/<Silva reference seqs QZA>
%env MYTAXA=$ref/<Silva taxonomy QZA>

# CPU threads
%env THREADS=8

env: CLUSTER_PER=99
env: MYREF=/home/sam/data/databases/silva/silva_132_97_16S.qza
env: REF_SEQS=/home/sam/data/databases/silva/Silva-ref-seqs.qza
env: MYTAXA=/home/sam/data/databases/silva/Silva_taxonomy.qza
env: THREADS=8


'base' is the main folder that will contain the analysis

'RAW' will contain the original sequencing files (fastq.gz)

'TRIM_OUT' contains the trimmed fastq files

'PANDA_OUT' contains the joined/merged fasta files

'FILTER_OUT' contains the filtered fasta files with renamed sequences

'CONVERT_OUT' contains the input for QIIME 2

'CHIMERA_OUT' contains the chimera checked sequences

'CLUSTER_OUT' contains the clustered OTU output

In [3]:
%env BASE=$base
%env TRIM_OUT=$base/trimmed
%env PANDA_OUT=$base/joined
%env FILTER_OUT=$base/filtered
%env CONVERT_OUT=$base/converted
%env CHIMERA_OUT=$base/chimera_checked
%env CLUSTER_OUT=$base/clustered
%env CLASSIFY_OUT=$base/classified


# Specific to classifying
%env REFSEQ=$ref/

env: BASE=/home/sam/analyses/20221118-rhodes-marine_mammals-16s-silva-original-99-percent
env: RAW=/home/sam/data/databases/silva/MiSeq_files_for_Gannet
env: TRIM_OUT=/home/sam/analyses/20221118-rhodes-marine_mammals-16s-silva-original-99-percent/trimmed
env: PANDA_OUT=/home/sam/analyses/20221118-rhodes-marine_mammals-16s-silva-original-99-percent/joined
env: FILTER_OUT=/home/sam/analyses/20221118-rhodes-marine_mammals-16s-silva-original-99-percent/filtered
env: CONVERT_OUT=/home/sam/analyses/20221118-rhodes-marine_mammals-16s-silva-original-99-percent/converted
env: CHIMERA_OUT=/home/sam/analyses/20221118-rhodes-marine_mammals-16s-silva-original-99-percent/chimera_checked
env: CLUSTER_OUT=/home/sam/analyses/20221118-rhodes-marine_mammals-16s-silva-original-99-percent/clustered
env: CLASSIFY_OUT=/home/sam/analyses/20221118-rhodes-marine_mammals-16s-silva-original-99-percent/classified
env: REFSEQ=/home/sam/data/databases/silva/


## Get list of samples

In [4]:
%%bash

mkdir --parents ${BASE}

if [[ -e ${BASE}/sample_ids.txt ]]; then
    rm ${BASE}/sample_ids.txt
fi

for file in ${RAW}/*R1*
do
    NAME=$(echo ${file} | awk -F "/" '{print $NF}' | awk -F ".R1" '{print $1}')
    echo ${NAME}
    echo ${NAME} >> ${BASE}/sample_ids.txt
done

SAMPLE_COUNT=$(wc ${BASE}/sample_ids.txt | awk '{print $1}')
echo -e "\nSample Count = ${SAMPLE_COUNT}"

310_S11_L001
311_S12_L001
312_S13_L001
313_S14_L001
314_S15_L001
315_S16_L001
316_S17_L001
317_S18_L001
318_S19_L001
319_S20_L001
320_S21_L001
Neg_S92_L001
Pos_S88_L001

Sample Count = 13


# Note: Missing adapter trimming FastA step - just for testing pipeline with new DBs

## Trimming - Trimmomatic
Trimmomatic Steps
1. Remove adapters
2. Remove leading/trailing low quality bases (quality score if 3 or lower, remove sequence)
3. Sliding window quality trimming (window of size 8 with average quality score of 10, remove if below)
4. Removing short reads (if length < 100)


Output:
- NAME.trimmed.paired.R1 : Sample trimmed paired R1 reads
- NAME.trimmed.paired.R2 : Sample trimmed paired R2 reads
- NAME.trimmed.unpaired.R1 : Sample trimmed broken R1 reads
- NAME.trimmed.unpaired.R2 : Sample trimmed broken R2 reads
- NAME.trimmomatic.log : Per sequence trimming information
- NAME.trimmomatic.stats.txt : Trimming stats for sample

In [5]:
%%bash

if [ ! -d ${TRIM_OUT} ]; then
    mkdir ${TRIM_OUT}
fi

for NAME in $(cat ${BASE}/sample_ids.txt)
do
    java -jar ${TRIMMOMATIC} \
        PE -phred33 \
        -threads ${THREADS} \
        -trimlog ${TRIM_OUT}/${NAME}.trimmomatic.log \
        ${RAW}/${NAME}_R1_001.fastq.gz \
        ${RAW}/${NAME}_R2_001.fastq.gz \
        ${TRIM_OUT}/${NAME}.trimmed.paired.R1.fq \
        ${TRIM_OUT}/${NAME}.trimmed.unpaired.R1.fq \
        ${TRIM_OUT}/${NAME}.trimmed.paired.R2.fq \
        ${TRIM_OUT}/${NAME}.trimmed.unpaired.R2.fq \
        ILLUMINACLIP:/home/sam/programs/Trimmomatic-0.39/adapters/NexteraPE-PE.fa:2:30:10 \
        LEADING:3 \
        TRAILING:3 \
        SLIDINGWINDOW:8:10 \
        MINLEN:100 \
        > ${TRIM_OUT}/${NAME}.trimmomatic.stats.txt 2>&1
done


## List the samples that passed trimming

In [6]:
%%bash

# Generate list of sample ids that passed trimming
if [ -f ${BASE}/sample_ids.pass_trimmed.txt ]; then
    rm ${BASE}/sample_ids.pass_trimmed.txt
fi

for file in $(ls -1 ${TRIM_OUT}/*trimmed.paired.R1.fq)
do
    NAME=$(echo ${file} | awk -F "/" '{print $NF}' | awk -F ".trimmed" '{print $1}')
    echo ${NAME}
    echo ${NAME} >> ${BASE}/sample_ids.pass_trimmed.txt
done

SAMPLE_COUNT=$(wc ${BASE}/sample_ids.pass_trimmed.txt | awk '{print $1}')
echo -e "\nSample Count = ${SAMPLE_COUNT}"

310_S11_L001
311_S12_L001
312_S13_L001
313_S14_L001
314_S15_L001
315_S16_L001
316_S17_L001
317_S18_L001
318_S19_L001
319_S20_L001
320_S21_L001
Neg_S92_L001
Pos_S88_L001

Sample Count = 13


## Joining - pandaseq

Output:
- NAME.joined.fa : Sample joined sequence fasta file
- NAME.joined.lengths.txt : Lengths of the sequences from the fasta file
- NAME.pandaseq.log : Log file from pandaseq

In [7]:
%%bash

if [ ! -d ${PANDA_OUT} ]; then
    mkdir ${PANDA_OUT}
fi

for NAME in $(cat ${BASE}/sample_ids.pass_trimmed.txt)
do
    pandaseq \
        -T 8 \
        -g ${PANDA_OUT}/${NAME}.pandaseq.log \
        -f ${TRIM_OUT}/${NAME}.trimmed.paired.R1.fq \
        -r ${TRIM_OUT}/${NAME}.trimmed.paired.R2.fq \
        -w ${PANDA_OUT}/${NAME}.joined.fa

    ./seq_dist.pl \
        ${PANDA_OUT}/${NAME}.joined.fa \
        > ${PANDA_OUT}/${NAME}.joined.lengths.txt
done

In [8]:
%%bash

# Generate list of samples that passed the joining step
if [ -f ${BASE}/sample_ids.pass_joined.txt ]; then
    rm ${BASE}/sample_ids.pass_joined.txt
fi

for file in $(ls -1 ${PANDA_OUT}/*joined.fa)
do
    NAME=$(echo ${file} | awk -F "/" '{print $NF}' | awk -F ".joined" '{print $1}')
    echo ${NAME}
    echo ${NAME} >> ${BASE}/sample_ids.pass_joined.txt
done

SAMPLE_COUNT=$(wc ${BASE}/sample_ids.pass_joined.txt | awk '{print $1}')
echo -e "\nSample Count = ${SAMPLE_COUNT}"

310_S11_L001
311_S12_L001
312_S13_L001
313_S14_L001
314_S15_L001
315_S16_L001
316_S17_L001
317_S18_L001
318_S19_L001
319_S20_L001
320_S21_L001
Neg_S92_L001
Pos_S88_L001

Sample Count = 13


# Stats for each sample after joining

In [9]:
%%bash

# Run the R script to generate the stats
./seq_stats.R ${PANDA_OUT}

              X1  X2  X3       X4  X5  X6
310_S11_L001 100 443 464 451.2833 465 595
311_S12_L001 100 443 460 454.1084 465 599
312_S13_L001 100 440 460 428.3542 465 599
313_S14_L001 100 459 465 455.9755 465 598
314_S15_L001 100 460 465 453.0552 465 599
315_S16_L001 100 440 464 435.2126 465 599
316_S17_L001 100 442 465 441.9699 465 597
317_S18_L001 100 459 464 441.4208 465 600
318_S19_L001 100 387 440 407.2018 465 599
319_S20_L001 100 440 464 451.2084 465 599
320_S21_L001 100 440 465 453.2890 465 598
Neg_S92_L001 100 464 465 460.5435 465 580
Pos_S88_L001 101 460 465 460.1400 465 595


# Filtering and renaming sequences

- Filtering
- Removing if
  - 7 or more N's
  - Homopolymers greater then 7 in length
  - Sequence length less than 400
  
  
- Need to rename the sequences to contain the sample ID

In [10]:
%%bash

# Setup the filter output folder
if [ ! -d ${FILTER_OUT} ]; then
    mkdir ${FILTER_OUT}
fi

# Loop through the samples and filter the fasta files
for NAME in $(cat ${BASE}/sample_ids.pass_joined.txt)
do
    ./seq_rename_and_filter.pl \
        ${PANDA_OUT}/${NAME}.joined.fa \
        ${NAME} \
        7 7 400 \
        > ${FILTER_OUT}/${NAME}.filtered.fa
done

In [11]:
%%bash

# Generate list of samples that passed the filtering step
if [ -f ${BASE}/sample_ids.pass_filtered.txt ]; then
    rm ${BASE}/sample_ids.pass_filtered.txt
fi

for file in $(ls -1 ${FILTER_OUT}/*.filtered.fa)
do
    NAME=$(echo ${file} | awk -F "/" '{print $NF}' | awk -F ".filtered" '{print $1}')
    echo ${NAME}
    echo ${NAME} >> ${BASE}/sample_ids.pass_filtered.txt
done

SAMPLE_COUNT=$(wc ${BASE}/sample_ids.pass_filtered.txt | awk '{print $1}')
echo -e "\nSample Count = ${SAMPLE_COUNT}"

310_S11_L001
311_S12_L001
312_S13_L001
313_S14_L001
314_S15_L001
315_S16_L001
316_S17_L001
317_S18_L001
318_S19_L001
319_S20_L001
320_S21_L001
Neg_S92_L001
Pos_S88_L001

Sample Count = 13


# Start of QIIME 2 Stuff
## Import fasta file into QIIME2 as a qza file

In [12]:
%%bash

if [ ! -d ${CONVERT_OUT} ]; then
    mkdir ${CONVERT_OUT}
fi

# Merge the filtered fasta files into a single file for import
cat ${FILTER_OUT}/*filtered.fa >> ${CONVERT_OUT}/all.sequences.filtered.fa

qiime \
    tools import \
    --input-path ${CONVERT_OUT}/all.sequences.filtered.fa \
    --output-path ${CONVERT_OUT}/all.sequences.filtered.qza \
    --type 'SampleData[Sequences]' \

Imported /home/sam/analyses/20221118-rhodes-marine_mammals-16s-silva-original-99-percent/converted/all.sequences.filtered.fa as QIIME1DemuxDirFmt to /home/sam/analyses/20221118-rhodes-marine_mammals-16s-silva-original-99-percent/converted/all.sequences.filtered.qza


## Dereplicate the qza file to generate the frequency table qza file

In [13]:
%%bash

qiime \
    vsearch dereplicate-sequences \
    --i-sequences ${CONVERT_OUT}/all.sequences.filtered.qza \
    --o-dereplicated-table ${CONVERT_OUT}/rep-seqs.table.qza \
    --o-dereplicated-sequences ${CONVERT_OUT}/rep-seqs.qza

Saved FeatureTable[Frequency] to: /home/sam/analyses/20221118-rhodes-marine_mammals-16s-silva-original-99-percent/converted/rep-seqs.table.qza
Saved FeatureData[Sequence] to: /home/sam/analyses/20221118-rhodes-marine_mammals-16s-silva-original-99-percent/converted/rep-seqs.qza


# Create reference file for clustering
## This is for a 97% clustered file from Silva_132 release
## SKIP THIS STEP IF THE REFERENCE FILE ALREADY EXISTS

In [19]:
%%bash

qiime tools import \
    --input-path ${RAW}/SILVA_138.1_SSURef_NR99_tax_silva.DNA.fasta \
    --output-path ${RAW}/SILVA_138.1_SSURef_NR99_tax_silva.DNA.qza \
    --type FeatureData[Sequence]

There was a problem importing /home/sam/data/databases/silva/raw/SILVA_138.1_SSURef_NR99_tax_silva.DNA.fasta:

  /home/sam/data/databases/silva/raw/SILVA_138.1_SSURef_NR99_tax_silva.DNA.fasta is not a(n) DNAFASTAFormat file:

  Multiple consecutive descriptions starting on line 563312



CalledProcessError: Command 'b'\nqiime tools import \\\n    --input-path ${RAW}/SILVA_138.1_SSURef_NR99_tax_silva.DNA.fasta \\\n    --output-path ${RAW}/SILVA_138.1_SSURef_NR99_tax_silva.DNA.qza \\\n    --type FeatureData[Sequence]\n'' returned non-zero exit status 1.

## OTU Clustering: either use open reference or closed reference.
## Both options are listed below

## Clustering with open reference

In [14]:
%%bash

if [ ! -d ${CLUSTER_OUT} ]; then
    mkdir ${CLUSTER_OUT}
fi

qiime \
    vsearch cluster-features-open-reference \
    --i-table ${CONVERT_OUT}/rep-seqs.table.qza \
    --i-sequences ${CONVERT_OUT}/rep-seqs.qza \
    --i-reference-sequences ${MYREF} \
    --p-perc-identity 0.${CLUSTER_PER} \
    --o-clustered-table ${CLUSTER_OUT}/table-or-${CLUSTER_PER}.qza \
    --o-clustered-sequences ${CLUSTER_OUT}/rep-seqs-or-${CLUSTER_PER}.qza \
    --o-new-reference-sequences ${CLUSTER_OUT}/new-ref-seqs-or-${CLUSTER_PER}.qza \
    --p-threads 0

Saved FeatureTable[Frequency] to: /home/sam/analyses/20221118-rhodes-marine_mammals-16s-silva-original-99-percent/clustered/table-or-99.qza
Saved FeatureData[Sequence] to: /home/sam/analyses/20221118-rhodes-marine_mammals-16s-silva-original-99-percent/clustered/rep-seqs-or-99.qza
Saved FeatureData[Sequence] to: /home/sam/analyses/20221118-rhodes-marine_mammals-16s-silva-original-99-percent/clustered/new-ref-seqs-or-99.qza


## Clustering with closed reference

In [15]:
%%bash

if [ ! -d ${CLUSTER_OUT} ]; then
    mkdir ${CLUSTER_OUT}
fi

qiime \
    vsearch cluster-features-closed-reference \
    --i-table ${CONVERT_OUT}/rep-seqs.table.qza \
    --i-sequences ${CONVERT_OUT}/rep-seqs.qza \
    --i-reference-sequences ${MYREF} \
    --p-perc-identity 0.${CLUSTER_PER} \
    --o-clustered-table ${CLUSTER_OUT}/table-cr-${CLUSTER_PER}.qza \
    --o-clustered-sequences ${CLUSTER_OUT}/rep-seqs-cr-${CLUSTER_PER}.qza \
    --o-unmatched-sequences ${CLUSTER_OUT}/unmatched-ref-seqs-cr-${CLUSTER_PER}.qza \
    --p-threads 0

Saved FeatureTable[Frequency] to: /home/sam/analyses/20221118-rhodes-marine_mammals-16s-silva-original-99-percent/clustered/table-cr-99.qza
Saved FeatureData[Sequence] to: /home/sam/analyses/20221118-rhodes-marine_mammals-16s-silva-original-99-percent/clustered/rep-seqs-cr-99.qza
Saved FeatureData[Sequence] to: /home/sam/analyses/20221118-rhodes-marine_mammals-16s-silva-original-99-percent/clustered/unmatched-ref-seqs-cr-99.qza


## Chimera Check

In [ ]:
%%bash

if [ ! -d ${CHIMERA_OUT} ]; then
    mkdir ${CHIMERA_OUT}
fi

qiime \
    vsearch uchime-denovo \
    --i-table ${CLUSTER_OUT}/table-or-${CLUSTER_PER}.qza \
    --i-sequences ${CLUSTER_OUT}/rep-seqs-or-${CLUSTER_PER}.qza \
    --o-chimeras ${CHIMERA_OUT}/chimeras.qza \
    --o-nonchimeras ${CHIMERA_OUT}/nonchimeras.qza \
    --o-stats ${CHIMERA_OUT}/chimera-check-stats.qza

## Use the non-chimeric sequences to exlude chimeras & borderline sequences from the clustered table & sequence files

In [ ]:
%%bash

qiime \
    feature-table filter-features \
    --i-table ${CLUSTER_OUT}/table-or-${CLUSTER_PER}.qza \
    --m-metadata-file ${CHIMERA_OUT}/nonchimeras.qza \
    --o-filtered-table ${CHIMERA_OUT}/table-nonchimeric-wo-borderline.qza

qiime \
    feature-table filter-seqs \
    --i-data ${CLUSTER_OUT}/rep-seqs-or-${CLUSTER_PER}.qza \
    --m-metadata-file ${CHIMERA_OUT}/nonchimeras.qza \
    --o-filtered-data ${CHIMERA_OUT}/rep-seqs-nonchimeric-wo-borderline.qza

qiime \
    feature-table summarize \
    --i-table ${CHIMERA_OUT}/table-nonchimeric-wo-borderline.qza \
    --o-visualization ${CHIMERA_OUT}/table-nonchimeric-wo-borderline.qzv

## Examine chimera filtering statistics

In [ ]:
%%bash

qiime metadata tabulate \
  --m-input-file ${CHIMERA_OUT}/chimera-check-stats.qza \
  --o-visualization ${CHIMERA_OUT}/chimera-check-stats.qzv

## Classify OTUs with VSEARCH consensus classifier using reference file

In [ ]:
%%bash

if [ ! -d ${CLASSIFY_OUT} ]; then
    mkdir ${CLASSIFY_OUT}
fi

qiime feature-classifier classify-consensus-vsearch \
    --i-query ${CHIMERA_OUT}/nonchimeras.qza \
    --i-reference-reads ${MYREF} \
    --i-reference-taxonomy ${REFSEQ}/${MYTAXA} \
    --p-maxaccepts 10 \
    --p-perc-identity 0.8 \
    --p-strand "both" \
    --p-min-consensus 0.51 \
    --p-threads ${THREADS} \
    --o-classification ${CLASSIFY_OUT}/rep-seqs-classified-by-Silva.qza \
    --output-dir ${CLASSIFY_OUT}/rep-seqs-unspecified-by-Silva